# <center>**Trabajo Practico 2 de Estructura de datos**
## **Búsqueda sobre indices:** 

### <font color=yellow>Diagrama de componentes UML :</font>
<img src="img/UML_EDDTP2COMP.png">

---

### <font color=yellow>Decisiones de diseño:</font>
- Se decidió crear 5 modulos para el funcionamiento del TP y otro para mostrar pruebas unitarias.
- Para estos 5 modulos previamente mencionados, no se utilizaron clases, si no que se implemento solo mediante funciones, debido a que no eran muchas y podia quedar legible de esta forma.
- Se agrego el modulo "Colorama" para darle "estilo" y mejorar la visibilidad de los errores.
- Se utilizó la clase UncompressedPostings proporcionada en el TP, para pasar la lista de apariciones a bytearray.
- Tambien se utilizó el algoritmo presentado en clase y en el libro de consulta, para la codificacion de longitud variable(Variable byte code).
- Debido a que en una prueba se tardo al importar modulos al inicio, se decidió importar solo cuando es necesario, luego de elegir opciones.
- Se decidió hacer uso a las expresiones regulares a la hora de llevar a cabo la busqueda binaria.
- En principio se pensó implementar una estructura similar a un HEAP, para el merge, pero se terminó usando un metodo para fusionar de a 2 dic y otro que lo regula, con recursividad.
- Se decidio que en la creacion de indice-posting list, se guarden los mapeos de termID,docID ademas de indices intermedios y el indice final mismo.
- Para la recoleccion se decidio agregar hora de recoleccion para poder tener en cuenta el tiempo utilizado, ademas de poner intervalos a los pedidos de informacion a las paginas.
- Se decidió que al ofrecer crear un indice invertido se pueda crear el indice tanto con posting list comprimida y no, todo esto con un boolean en la funcion.
- En refencia a la busqueda, se interpretó el enunciado del TP de forma que quedaba a libre eleccion el usar indice comprimido o no para llevar a cabo esta misma. Por esto mismo se decidió hacer la busqueda sobre el indice comprimido.
- Frente a la incerteza de utilizar una interfaz en la consola, se prefirio hacer un menú simple con las 3 opciones, de las cuales derivan diferentes pedidos.
- Se decidió que se pueda presionar Ctrl+C para salir de la ejecucion del recolector de noticias.
- Para la guardar datos en disco se uso:
    - shelve para tener 2 diccionarios en un solo archivo. 
    - Tambien pickle para los casos que se necesitaba guardar solo un indice.
    - Y json para guardar indice sobre la posting list, debido a que no era necesario que los datos se guarden en binario
- Se decidió agregar color a los mensajes de error. En primer lugar se utilizo un modulo llamado "Colorama", pero luego para evitar uso de este usó códigos de secuencias o códigos de escape del estándar ANSI para cambiar los formatos de las salidas. Colorama se usará solo para dar "estilo".
- A algunos métodos se les agrego un guion bajo adelante para "simular" o seguir la convención de que es privado
---

### <font color=yellow>Descripción breve de cada archivo fuente y funciones:</font>
- ***App.py***:
     - ejecutar_opciones: Se encargará de recibir el numero y evaluar que funcion llevar a cabo.
     - ejecutar_recolector: Será utilizada para llamar funciones del modulo Recolector_de_noticias. Es llamada por "ejecutar_opciones".
     - main: se utilizará para mostrar las opciones disponibles, y que se puede reutilizar. Esta funcion llamará a "ejecutar_opciones".
- ***Recolector_de_noticias.py***:
     - _compara_xmls_y_añade_articulos_nuevos:Recibirá una url y una direccion de archivo xml.Pasará los datos a una estructura de arbol y comprobara que no se repitan los textos de dos elementos(titulo y fecha).Se avisa sobre estructuras mal formadas en paginas y se pone tiempo al pedido.
     - _crear_xml_mediante_config_y_agrega_articulo: recibe como parametro el path donde se debe encontrar el archivo de configuracion .ini, con nombre del canal como seccion, url_base y direcciones recortadas de las diferentes secciones periodisticas. Con este archivo, verificará que exista la carpeta propia al canal y en caso de que no sea así, se creará una y luego se pasará a recolectar noticias sobre las secciones del canal.
     - recolectar_articulos_segun_config: clase que funcionara solo como envoltorio, y se hara uso para agarrar una excepcion
- ***Indice_invertido_y_compresion.py***:
    - Variable_byte_encode_number: estas 3 funciones, se implementaron a partir del pseudocodigo. Esta funcion se encarga de codificar el numero.
    - Variable_byte_encode_list: esta codificará la lista de numeros.
    - Variable_byte_decode_list: decodificará la lista de numeros. Utilizado para la compresion de posting list.
    - _generar_saltos_docID: Recibirá un indice con posting list sin comprimir y hara el salto entre docIDs, mediante una especie de resta consecutiva.
    - _comprimir_posting_list: rebice path para crear indice de las apariciones, se aplican los saltos y luego se procede a iterar para comprimir las listas de aparicion. Tambien se guardan los datos para mejorar la eficiencia al generar el indice invertido.
    - crear_indice_invertido: Recibe path de la carpeta de medios para poder crear el indice, además da la opcion de crear indice invertido sobre posting list comprimida o no. 
- **Crear_indice_post_list.py**:
    - clase UncompressedPostings
        - encode: codifica una lista a bytearray.
        - decode: decodifica el bytearray a una lista.
    - _guardar_indices_intermedios_o_dic: se utilizará para guardar el mapeo de termID y docID, o los indices intermedios diferenciandolos por n° de bloque.
    - acondicionar_palabra: Cambia las letras con tilde, "sacandolas" y quitando los signos de puntuacion.
    - _merge_indices_intermedios: Toma una lista con indices de posting list e itera sobre estos para pasarlos a la proxima funcion, la cual fusionará los indices.
    - _merge_indices: Toma dos indices y los fusiona, utilizando colas para luego evaluar termIDs y luego decodificar para fusionar sets o agregar listas al indice como resultado de la fusion.
    - BSBI_algorithm: Un intento de implementacion. Reciba la carpeta donde se encuentran los canales, y trabaja los archivos con secciones periodisticas, devolviendo un indice con estructura-> termID: [posting list]. esta posting list como byteArray
- ***Busqueda.py***:
    - _buscar_e_imprimir_noticia_de_doc: Recibe una palabra y un nombre de un doc, lo pasa a arbol e itera los titulos de este, buscando con expresiones regulares la aparicion de la palabra, en caso de encontrarlo imprime, canal-medio-titulo-fecha
    - busqueda_booleana: Recibe una lista de palabras separadas por blanco y una ruta donde se encuentran los canales. Si no existe el indice invertido, creará uno. Luego recuperará mapeos y se utilizara la funcion para buscar.
- ***Excepciones_TP2.py***:
    - clase Indices_y_busqueda_exception
        - sera la super clase de excepciones propias
    - clase Tipo_de_archivo_incorrecto_exception
        - constructor: tomará el nombre del archivo, para devolver un mensaje mas claro
        - en caso de que el archivo no tenga extension .ini
    - clase Opcion_incorrecta_exception
        - constructor: tomará la opcion elegida y las que son validas, para mejorar el uso.
        - __str__: mostrara el error correspondiente a opcion incorrecta
---
#### Se agrega parte del codigo a ejecutar:
- Un aspecto a tener en cuenta:
    - Se hicieron pruebas sobre la carpeta "Medios_prueba" y funciona correctamente, la cual contiene unos 138mb.A la carpeta Medios habria que borrarle la subcarpeta de "20minutos".


In [ ]:
import time;import configparser
from colorama import init,Style;init()
import os
from datetime import datetime
from Excepciones.Excepciones import *
color_error="\033[4;31m"
color_blanco='\033[0;37m'
def ejecutar_recolector(config):
    if not config[-4:] == ".ini":
        raise Tipo_de_archivo_incorrecto_exception(config)
    if not os.path.exists(config):
        raise FileNotFoundError
    from src.Recolector_de_noticias import recolectar_articulos_segun_config
    ruta_medios=input("Ingrese el path de la carpeta con los Medios: ")
    try:
        configuracion = configparser.ConfigParser()
        configuracion.read(config)
        tiempo_espera_entre_ejecuciones=configuracion["DEFAULT"]["query_interval"]
        while True:
            recolectar_articulos_segun_config(ruta_medios,config)
            print(f"Articulos guardados. {datetime.now().hour}:{datetime.now().minute}")
            time.sleep(int(tiempo_espera_entre_ejecuciones))#10 minutos   300=5 min
    except KeyboardInterrupt:
        print(color_error+"Se cerrara el programa")
        time.sleep(5)
    except KeyError:
        print(color_error + Style.BRIGHT + 'Asegurese de que el archivo tenga el apartado "query_interval"')
        print("Volverá al menú principal."+color_blanco)
        main()
def ejecutar_opciones(opcion_elegida):
    if opcion_elegida == 1:
        print(color_blanco+"Presione Ctrl+C para detener la recoleccion de articulos en cualquier momento")
        ruta_config=input("Introduzca la ruta de la configuracion(debe contener intervalo de tiempo, url base y direcciones de rss, ext ini):\n")
        try:
            ejecutar_recolector(ruta_config)
        except Tipo_de_archivo_incorrecto_exception as e:
            print(e)
        except FileNotFoundError:
            print(color_error+f"El archivo {ruta_config} no fue encontrado."+color_blanco)
    elif opcion_elegida == 2:#se va a dar a elegir si crear indice con compresion o sin
        from src.Indice_invertido_y_compresion import crear_indice_invertido
        ruta_canales=input("Introduzca la ruta donde se encuentran los canales, para crear indice y luego invertir: ")
        compresion=input("Introduzca 'si o no' en caso de querer un indice comprimido: ")
        if compresion.lower() == "si":
            crear_indice_invertido(ruta_canales,True)
        elif compresion.lower() == "no":
            crear_indice_invertido(ruta_canales,False)
        else:
            raise Opcion_incorrecta_exception(compresion,"si-no")
    elif opcion_elegida == 3:
        from src.Busqueda import busqueda_booleana
        palabras_a_buscar=input("Ingrese las palabras a buscar, separadas por un espacio: ")
        ruta_canales=input("Introduzca la ruta donde se encuentran los canales, para crear indice en caso de que no exista: ")
        busqueda_booleana(palabras_a_buscar,ruta_canales)
        time.sleep(15)
    else:
        raise Opcion_incorrecta_exception(opcion_elegida,"1-2-3")

def main():
    try:
        opcion_elegida=int(input("Escriba el numero de la funcion a utilizar:\n1-Recolectar Noticias.\n2-Crear indice "
                                 "invertido con compresion o no.\n3-Realizar busqueda de palabras.\n"))
        try:
            ejecutar_opciones(opcion_elegida)
        except Opcion_incorrecta_exception as e:
            print(e)
        print(color_blanco)
        utilizar_otra_funcion=input("En caso de querer utilizar otra funcion, introduzca 'si', cualquier cosa en caso contrario: ")
        if utilizar_otra_funcion.lower() == "si":
            main()
    except ValueError:
        print("\033[4;31m" + Style.BRIGHT + "Debe ingresar un entero para elegir una Opcion")
        time.sleep(5)

if __name__=="__main__":
    main()

Escriba el numero de la funcion a utilizar:
1-Recolectar Noticias.
2-Crear indice invertido con compresion o no.
3-Realizar busqueda de palabras.
 3
Ingrese las palabras a buscar, separadas por un espacio:  macri
Introduzca la ruta donde se encuentran los canales, para crear indice en caso de que no exista:  Medios


---------INDICE RECUPERADO----------
La palabra 'macri'', fue encontrada en:
iProfesional/economía.xml Cerró la pizzería que Macri visitó hace un año con su hija Antonia: qué les dijo en ese momento a los dueños Wed, 25 Sep 2019 09:10:00 -0300
iProfesional/economía.xml Macri habló con el N°1 del FMI para destrabar los u$s5.400 M, pero la negociación seguirá en octubre Tue, 24 Sep 2019 15:34:00 -0300
iProfesional/economía.xml Macri, sobre el bono de $5.000: "Es importante que todos hagamos nuestro aporte" Tue, 24 Sep 2019 11:56:00 -0300
iProfesional/economía.xml El índice de pobreza rondará el 35% y le pone presión adicional al cierre de la campaña macrista Fri, 27 Sep 2019 06:50:00 -0300
iProfesional/economía.xml El índice de pobreza rondará el 35% y le pone presión adicional al cierre de la campaña macrista Fri, 27 Sep 2019 03:50:00 -0300
iProfesional/economía.xml Alberto Fernández le respondió a Mauricio Macri: "Lo único que generó fueron pobres" Sat, 28 Sep 2019 22:18:00 -0300
iProf

### <font color=yellow>Ejemplo de uso::</font>
**Al ejecutar App saltará el menú, se elegirá la opcion 1 y se introducirá "config.ini" y "Medios" posteriormente, para recolectar noticias**
<img src="capturas de pantalla/1.png">
**Luego de recolectar una vez, con ctrl+c, pararemos el recolector, para despues ingresar "si" y poder elegir otra funcionalidad**
<img src="capturas de pantalla/2.png">
**Se elegirá la opcion de crear indice invertido con "2", se ingresará "Medios" y luego "si" para comprimir el indice**
<img src="capturas de pantalla/3.png">
**Continuamos con otra funcionalidad, la opcion 3, ingresamos 3 palabras para la busqueda: "fernandez macri adoquin" y "Medios" por las dudas de que no exista el indice**
<img src="capturas de pantalla/4.png">
**algunos resultados mas de la misma busqueda**
<img src="capturas de pantalla/5.png">
<img src="capturas de pantalla/6.png">
<font color=green>Agrego algunos casos de error en el recolector.</font>
<img src="capturas de pantalla/7.png">
<img src="capturas de pantalla/8.png">
<img src="capturas de pantalla/9.png">

---
### <font color=yellow>Conclusión:</font>
Resultó interesante el trabajo propuesto.Quizá se complicó un poco al momento de comprender el problema.Pero la parte del codigo no fue tan complicada.Hubiese estado bueno poder tratar de mejorar un poco el programa, se pensó en un momento de hacer un thread para el recolector de noticias, pero al no saber si seria correcto, se prefirió no hacerlo.